In [7]:
!python ..\src\GUI.py

Transformed data for prediction:
   ACT composite score  ...  Parental_education_encoded
0                   36  ...                           2

[1 rows x 8 columns]
Scaled data:
[[ 2.66623519 -2.75786203 -0.34666577 -0.92347383 -3.15906098 -3.27553313
  -0.69478265]]
Transformed data for prediction:
   ACT composite score  ...  Parental_education_encoded
0                   36  ...                           2

[1 rows x 8 columns]
Scaled data:
[[ 2.66623519 -2.75786203 -0.34666577 -0.92347383 -3.15906098 -3.27553313
   0.01273532]]
Transformed data for prediction:
   ACT composite score  ...  Parental_education_encoded
0                   36  ...                           2

[1 rows x 8 columns]
Scaled data:
[[ 2.66623519 -2.75786203 -0.34666577 -0.92347383 -3.15906098 -3.27553313
  -0.69478265]]
Transformed data for prediction:
   ACT composite score  ...  Parental_education_encoded
0                   36  ...                           2

[1 rows x 8 columns]
Scaled data:
[[ 2.66623